### 0 Important Libraries

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import os
import rarfile
import pandas as pd
from tqdm import tqdm
import re
from pyvis.network import Network

### 1 Extraction of Data from RAR Files

In [2]:
def extract_file(rar_file, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    with rarfile.RarFile(rar_file) as rf:
        for f in tqdm(rf.infolist(), desc='Extracting files', unit='file'):
            rf.extract(f, output_dir)

os.makedirs('raw-data', exist_ok=True)

In [3]:
# Create a folder named 'renamed-data' to store the renamed files
if not os.path.exists('renamed-data'):
    os.makedirs('renamed-data')

provinces = ['Gilan', 'Golestan', 'Mazandaran']

for province in provinces:
    # Create folders Hourly and Daily
    if not os.path.exists('renamed-data/' + province):
        os.makedirs('renamed-data/' + province)
    if not os.path.exists('renamed-data/' + province + '/Hourly'):
        os.makedirs('renamed-data/' + province + '/Hourly')
    if not os.path.exists('renamed-data/' + province + '/Daily'):
        os.makedirs('renamed-data/' + province + '/Daily')

In [4]:
# Loop through all the files in the compressed-data folder and extract .rar files
for file in tqdm(os.listdir('compressed-data')):
    if file.endswith('.rar'):
        extract_file(os.path.join('compressed-data', file), f'raw-data/{file.replace(".rar", "")}')

  0%|          | 0/44 [00:00<?, ?it/s]

100%|██████████| 44/44 [00:26<00:00,  1.67it/s]


### 2 Data Accumulation

In [5]:
# Create a folder named accumulated-data
os.makedirs('accumulated-data', exist_ok=True)

names_english = ['Gilan', 'Mazandaran', 'Golestan']
names_persian = ['گيلان', 'مازندران', 'گلستان']

# In the accumulated-data folder, create a folder for each province
for name in names_english:
    os.makedirs(f'accumulated-data/{name}', exist_ok=True)

for folder in tqdm(os.listdir('raw-data')):
    # If the folder name contains the name of a province in Persian or English case-insensitively
    # Move the folder to the corresponding province folder in the accumulated-data folder
    for i in range(3):
        if names_english[i].lower() in folder.lower() or names_persian[i] in folder:
            os.rename(f'raw-data/{folder}', f'accumulated-data/{names_english[i]}/{folder}')

# If raw-data folder is empty, remove it
if not os.listdir('raw-data'):
    os.rmdir('raw-data')

100%|██████████| 44/44 [00:00<00:00, 4853.37it/s]


In [6]:
# Create a folder named converted data
os.makedirs('converted-data', exist_ok=True)

# Loop through all the folders in the accumulated-data folder
for folder in tqdm(os.listdir('accumulated-data')):
    # For each folder, create a folder with the same name in the converted-data folder
    os.makedirs(f'converted-data/{folder}', exist_ok=True)
    # Create a Daily and Hourly folder inside the folder
    os.makedirs(f'converted-data/{folder}/Daily', exist_ok=True)
    os.makedirs(f'converted-data/{folder}/Hourly', exist_ok=True)

    for root, _, files in os.walk(f'accumulated-data/{folder}'):
        for file in files:
            # If the file is an excel file
            if file.endswith('.xlsx'):
                # Read the excel file
                df = pd.read_excel(os.path.join(root, file))
                # Set the first row as the column names
                df.columns = df.iloc[0]
                df = df.drop(0)
                # If the name contains 'daily' case-insensitively
                if 'daily' in file.lower():
                    # Check if there is file with the same name in the Daily folder
                    if os.path.exists(f'converted-data/{folder}/Daily/{file.replace(".xlsx", ".csv")}'):
                        # Read the file in the Hourly folder
                        df_hourly = pd.read_csv(f'converted-data/{folder}/Daily/{file.replace(".xlsx", ".csv")}')
                        # Concatenate the two dataframes
                        df = pd.concat([df_hourly, df], ignore_index=True)
                    # Save the file in the Daily folder
                    df.to_csv(f'converted-data/{folder}/Daily/{file.replace(".xlsx", ".csv")}', index=False)
                # If the name contains 'hourly' case-insensitively
                elif 'hourly' in file.lower():
                    # Check if there is file with the same name in the Hourly folder
                    if os.path.exists(f'converted-data/{folder}/Hourly/{file.replace(".xlsx", ".csv")}'):
                        # Read the file in the Hourly folder
                        df_hourly = pd.read_csv(f'converted-data/{folder}/Hourly/{file.replace(".xlsx", ".csv")}')
                        # Concatenate the two dataframes
                        df = pd.concat([df_hourly, df], ignore_index=True)
                    # Save the file in the Hourly folder
                    df.to_csv(f'converted-data/{folder}/Hourly/{file.replace(".xlsx", ".csv")}', index=False)

100%|██████████| 3/3 [03:53<00:00, 77.72s/it]


In [10]:
# Make renamed-data
os.makedirs('renamed-data', exist_ok=True)

In [11]:
# Copy folder ./converted-data and its content to ./renamed-data
os.system('cp -r converted-data/* renamed-data')

0

### 3 Entity Formation

In [13]:
# Inside converted-date folder
provinces = ['Gilan', 'Golestan', 'Mazandaran']
for province in provinces:
    path = f'./renamed-data/{province}'
    for filename in os.listdir(path + '/Daily'):
        temp = re.sub(r'Daily \d{6}', '', filename)
        temp = re.sub(r'[\u202c\u202b\u200c]', '', temp)
        temp = re.sub(r'.csv', '', temp).strip()

        os.rename(f'{path}/Daily/{filename}', f'{path}/Daily/{temp}.csv')
    for filename in os.listdir(path + '/Hourly'):
        temp = re.sub(r'Hourly \d{6}', '', filename)
        temp = re.sub(r'[\u202c\u202b\u200c]', '', temp)
        temp = re.sub(r'.csv', '', temp).strip()
        os.rename(f'{path}/Hourly/{filename}', f'{path}/Hourly/{temp}.csv')

In [26]:
# Create mapped-data directory with provinces inside
os.makedirs('mapped-data', exist_ok=True)
for province in provinces:
    os.makedirs(f'mapped-data/{province}', exist_ok=True)
    os.makedirs(f'mapped-data/{province}/Daily', exist_ok=True)
    os.makedirs(f'mapped-data/{province}/Hourly', exist_ok=True)

In [597]:
# For each filename in ./mapped-data/Gilan/Daily
for filename in os.listdir('./renamed-data/Golestan/Daily'):
    print(filename)
    temp = filename
    # Remove .csv
    temp = re.sub(r'.csv', '', temp)
    # Turn to array by splitting using -, (, )
    temp = re.split(r'[-(),]', temp)
    # Trim elements of the array
    temp = [x.strip() for x in temp]
    # Remove empty elements
    temp = list(filter(None, temp))
    print(temp)

    # Ask for input
    print('Enter the following information')
    start = input('Start city: ')
    end = input('End city: ')
    # If start or end is empty continue
    # If start equals -1 break
    if start == '-1':
        break
    if not start or not end:
        continue
    # If start and end are single characters
    if len(start) == 1:
        start = int(start)
        start = temp[start]
    if len(end) == 1:
        end = int(end)
        end = temp[end]

    more = input('Need more: ')
    
    # If length of temp is more than 2
    if len(temp) > 2 or more == 'y':
        # If start is not in temp
        edge = input('Enter the edge: ')
        # If edge contains comma
        if ',' in edge:
            edge = edge.split(',')
            # For each element in edge
            for i in range(len(edge)):
                # Trim the element
                edge[i] = temp[int(edge[i].strip())]
            edge = [x.strip() for x in edge]
            # Join using comma
            edge = ' - '.join(edge)
    # Renamed ./mapped-data/Gilan/Daily/filename to ./mapped-data/Gilan/Daily/start - end.csv
    if len(temp) > 2:
        # Copy the file to from renamed-data to mapped-data
        os.system(f'cp "./renamed-data/Golestan/Daily/{filename}" ./mapped-data/Golestan/Daily/')
        os.rename(f'./mapped-data/Golestan/Daily/{filename}', f'./mapped-data/Golestan/Daily/{start} - {end} ({edge}).csv')
        os.system(f'cp "./renamed-data/Golestan/Hourly/{filename}" ./mapped-data/Golestan/Hourly/')
        os.rename(f'./mapped-data/Golestan/Hourly/{filename}', f'./mapped-data/Golestan/Hourly/{start} - {end} ({edge}).csv')
    else:
        # os.rename(f'./mapped-data/Gilan/Daily/{filename}', f'./mapped-data/Gilan/Daily/{start} - {end}.csv')
        # os.rename(f'./mapped-data/Gilan/Daily/{filename}', f'./mapped-data/Gilan/Hourly/{start} - {end}.csv')
        os.system(f'cp "./renamed-data/Golestan/Daily/{filename}" ./mapped-data/Golestan/Daily/')
        os.rename(f'./mapped-data/Golestan/Daily/{filename}', f'./mapped-data/Golestan/Daily/{start} - {end}.csv')
        os.system(f'cp "./renamed-data/Golestan/Hourly/{filename}" ./mapped-data/Golestan/Hourly/')
        os.rename(f'./mapped-data/Golestan/Hourly/{filename}', f'./mapped-data/Golestan/Hourly/{start} - {end}.csv')



شاهرود - آزادشهر.csv
['شاهرود', 'آزادشهر']
Enter the following information
کردکوي - بندر ترکمن.csv
['کردکوي', 'بندر ترکمن']
Enter the following information
گنبد - آق قلا.csv
['گنبد', 'آق قلا']
Enter the following information
مينودشت - گنبد.csv
['مينودشت', 'گنبد']
Enter the following information
بندرترکمن - آق قلا.csv
['بندرترکمن', 'آق قلا']
Enter the following information
اينچه برون - مرز پل.csv
['اينچه برون', 'مرز پل']
Enter the following information
گرگان - کردکوي.csv
['گرگان', 'کردکوي']
Enter the following information
گاليکش - مينودشت.csv
['گاليکش', 'مينودشت']
Enter the following information
بندر ترکمن - سه راهي يساقي.csv
['بندر ترکمن', 'سه راهي يساقي']
Enter the following information
آزادشهر - گنبد.csv
['آزادشهر', 'گنبد']
Enter the following information
آق قلا - گنبد.csv
['آق قلا', 'گنبد']
Enter the following information
آزادشهر - مينودشت.csv
['آزادشهر', 'مينودشت']
Enter the following information
گرگان - علي آباد.csv
['گرگان', 'علي آباد']
Enter the following information
کردکوي - گر

In [5]:
gilan_translation = {
  'سياهکل': 'Siahkal',
  'سنگر': 'Sangar',
  'ضيابر': 'Ziabar',
  'پونل': 'Punel',
  'اردبيل': 'Ardabil',
  'آستارا': 'Astara',
  'خلخال': 'Khalkhal',
  'انزلي': 'Anzali',
  'رضوانشهر': 'Rezvanshahr',
  'امامزاده هاشم': 'Imamzadeh Hashem',
  'سراوان': 'Saravan',
  'رودبار': 'Rudbar',
  'رستم آباد': 'Rostamabad',
  'کوچصفهان': 'Kuchesfahan',
  'رشت': 'Rasht',
  'لوشان': 'Lowshan',
  'قزوین': 'Ghazvin',
  'حسن رود': 'Hasanrud',
  'فومن': 'Fuman',
  'بازارجمعه': 'Bazar Jomeh',
  'کياشهر': 'Kiashahr',
  'خمام': 'Khomam',
  'پيربازار': 'Pirbazar',
  'ماسوله': 'Masouleh',
  'لاکان': 'Lakan',
  'تالش': 'Talesh',
  'شاقاجي': 'Shaqaji',
  'چابکسر': 'Chaboksar',
  'کلاچاي': 'Kalachay',
  'جيرده': 'Jirdeh',
  'اسالم': 'Asalem',
  'طارم': 'Tarom',
  'منجيل': 'Manjil',
  'املش': 'Amlash',
  'رودسر': 'Rudsar',
  'آستانه': 'Astaneh',
  'ديلمان': 'Deylaman',
  'لاهيجان': 'Lahijan',
  'گيلوان': 'Gilvan',
  'رامسر': 'Ramsar',
  'لنگرود': 'Langarud',
  'صومعه سرا': 'Someh Sara',
  'آق چاي': 'Aq Chay',
  'مرز استان': 'Gilan Border',
  'کلان کلايه': 'Kalan Kalaye'
}

import pandas as pd
# Create a dataframe using the translation dictionary
df = pd.DataFrame(list(gilan_translation.items()), columns=['Persian', 'English'])

# Add a column "Province" and set it to 'Gilan'
df['Province'] = 'Gilan'
df

Persian           English Province
0          سياهکل           Siahkal    Gilan
1            سنگر            Sangar    Gilan
2           ضيابر            Ziabar    Gilan
3            پونل             Punel    Gilan
4          اردبيل           Ardabil    Gilan
5          آستارا            Astara    Gilan
6           خلخال          Khalkhal    Gilan
7           انزلي            Anzali    Gilan
8        رضوانشهر       Rezvanshahr    Gilan
9   امامزاده هاشم  Imamzadeh Hashem    Gilan
10         سراوان           Saravan    Gilan
11         رودبار            Rudbar    Gilan
12      رستم آباد        Rostamabad    Gilan
13       کوچصفهان       Kuchesfahan    Gilan
14            رشت             Rasht    Gilan
15          لوشان           Lowshan    Gilan
16          قزوین           Ghazvin    Gilan
17        حسن رود          Hasanrud    Gilan
18           فومن             Fuman    Gilan
19      بازارجمعه       Bazar Jomeh    Gilan
20         کياشهر          Kiashahr    Gilan
21           خمام            Khomam    Gilan
22       پيربازار          Pirbazar    Gilan
23         ماسوله          Masouleh    Gilan
24          لاکان             Lakan    Gilan
25           تالش            Talesh    Gilan
26         شاقاجي           Shaqaji    Gilan
27         چابکسر         Chaboksar    Gilan
28         کلاچاي          Kalachay    Gilan
29          جيرده            Jirdeh    Gilan
30          اسالم            Asalem    Gilan
31           طارم             Tarom    Gilan
32          منجيل            Manjil    Gilan
33           املش            Amlash    Gilan
34          رودسر            Rudsar    Gilan
35         آستانه           Astaneh    Gilan
36         ديلمان          Deylaman    Gilan
37        لاهيجان           Lahijan    Gilan
38         گيلوان            Gilvan    Gilan
39          رامسر            Ramsar    Gilan
40         لنگرود          Langarud    Gilan
41      صومعه سرا        Someh Sara    Gilan
42         آق چاي           Aq Chay    Gilan
43      مرز استان      Gilan Border    Gilan
44     کلان کلايه      Kalan Kalaye    Gilan

In [589]:
road_translation = {
  'آستارا - اردبيل': 'Astara-Ardabil Rd',
  'رشت - قزوين': 'Rasht-Ghazvin FWY',
  'قزوين - رشت': 'Ghazvin-Rasht FWY',
  'آزادراه': 'FWY',
  'قدیم': 'Old Rd',
  'فومن': 'Fuman Rd',
  'سراوان': 'Saravan Rd',
  'مرز استان': 'Gilan Exit Rd',
  'اسالم': 'Asalem Road',
  'منجيل - طارم': 'Manjil-Tarom Rd',
  'شلمان': 'Shalman Road',
  'طارم - منجيل': 'Tarom-Manjil Rd',
  'اردبيل - آستارا': 'Ardabil-Astara Rd',
  '': '',
}

In [20]:
year = '1400'
month = '01'
day = '16'

In [8]:
translation = {
'کردکوي': 'Kordkuy',
'بندر ترکمن': 'Bandar Torkaman',
'گرگان': 'Gorgan',
'گلوگاه': 'Galugah',
'سه راهي يساقي': 'Se-rah-e Yasaqi',
'آق قلا': 'Aq Qala',
'گميشان': 'Gomishan',
'گنبد': 'Gonbad',
'مينودشت': 'Minudasht',
'آزادشهر': 'Azadshahr',
'اينچه برون': 'Incheh Borun',
'سه راهي کلاله': 'Se-rah-e Kalaleh',
'گاليکش': 'Galikesh',
'خوش ییلاق': 'Khosh Yeylagh',
'علي آباد': 'Aliabad',
'شاهرود': 'Shahroud',
'تنگراه': 'Tangrah',
'جنگل گلستان': 'Golestan Forest',
'کلاله': 'Kalaleh',
'مرزپل': 'Marzpol',
'مراوه تپه': 'Maraveh Tappeh',
}

df3 = pd.DataFrame(list(translation.items()), columns=['Persian', 'English'])
df3['Province'] = 'Golestan'
df3

Persian           English  Province
0          کردکوي           Kordkuy  Golestan
1      بندر ترکمن   Bandar Torkaman  Golestan
2           گرگان            Gorgan  Golestan
3          گلوگاه           Galugah  Golestan
4   سه راهي يساقي   Se-rah-e Yasaqi  Golestan
5          آق قلا           Aq Qala  Golestan
6          گميشان          Gomishan  Golestan
7            گنبد            Gonbad  Golestan
8         مينودشت         Minudasht  Golestan
9         آزادشهر         Azadshahr  Golestan
10     اينچه برون      Incheh Borun  Golestan
11  سه راهي کلاله  Se-rah-e Kalaleh  Golestan
12         گاليکش          Galikesh  Golestan
13      خوش ییلاق     Khosh Yeylagh  Golestan
14       علي آباد           Aliabad  Golestan
15         شاهرود          Shahroud  Golestan
16         تنگراه           Tangrah  Golestan
17    جنگل گلستان   Golestan Forest  Golestan
18          کلاله           Kalaleh  Golestan
19          مرزپل           Marzpol  Golestan
20      مراوه تپه    Maraveh Tappeh  Golestan

In [43]:
translation = {
  'کياکلا': 'Kiyakala',
  'بابل': 'Babol',
  'گتاب': 'Gatab',
  'آمل': 'Amol',
  'قائمشهر': 'Qaemshahr',
  'فريدونکنار': 'Fereydunkenar',
  'بابلسر': 'Babolsar',
  'شاه زيد': 'Shahzeyd',
  'سه راه سنگچال': 'Se-rah-e Sangchal',
  'سه راه بلده': 'Se-rah-e Baladeh',
  'پارک جنگلي': 'Park Jangali',
  'کياسر': 'Kiasar',
  'ساري': 'Sari',
  'گرماب': 'Garmab',
  'سمنان': 'Semnan',
  'جويبار': 'Juybar',
  'نکا': 'Neka',
  'خزرآباد': 'Khazarabad',
  'شکتا': 'Shakta',
  'بهنمير': 'Behnamir',
  'زرگرشهر': 'Zargarshahr',
  'بهشهر': 'Behshahr',
  'بندر اميرآباد': 'Bandar-e Amirabad',
  'گلوگاه': 'Galugah',
  'کردکوي': 'Kordkuy',
  'زاغمرز': 'Zagheh Marz',
  'دابودشت': 'Dabudsht',
  'محمودآباد': 'Mahmudabad',
  'شيرگاه': 'Shirgah',
  'پل سفيد': 'Pol-e Sefid',
  'گرزين خيل': 'Garzin Kheil',
  'گدوک': 'Gaduk',
  'ورسک': 'Veresk',
  'نوشهر': 'Nowshahr',
  'چالوس': 'Chalus',
  'سی سنگان': 'Sisangan',
  'مرزن آباد': 'Marzanabad',
  'کلارآباد': 'Kalarabad',
  'عباس آباد': 'Abbasabad',
  'دوآب': 'Doab',
  'مکارود': 'Makarud',
  'سوادکوه': 'Savadkuh',
  'فيروزکوه': 'Firuzkuh',
  'کلاردشت': 'Kalardasht',
  'تنکابن': 'Tonekabon',
  'نور': 'Nur',
  'پنجاب': 'Panjab',
  'گزنک': 'Gazanak',
  'پلور': 'Polur',
  'کندوان': 'Kandovan',
  'سياه بيشه': 'Siah Bisheh',
  'هزارچم': 'Hazarcham',
  'رامسر': 'Ramsar',
  'چابکسر': 'Chaboksar',
  'امامزاده هاشم تهران': 'Imamzadeh Hashem Tehran'
}

# df2 = pd.DataFrame(list(translation_mazandaran.items()), columns=['Persian', 'English'])

# df2['Province'] = 'Mazandaran'
# df2

In [10]:
# Append df, df2, and df3
df = pd.concat([df, df2, df3], ignore_index=True)

In [11]:
df

Persian          English  Province
0         سياهکل          Siahkal     Gilan
1           سنگر           Sangar     Gilan
2          ضيابر           Ziabar     Gilan
3           پونل            Punel     Gilan
4         اردبيل          Ardabil     Gilan
..           ...              ...       ...
116       تنگراه          Tangrah  Golestan
117  جنگل گلستان  Golestan Forest  Golestan
118        کلاله          Kalaleh  Golestan
119        مرزپل          Marzpol  Golestan
120    مراوه تپه   Maraveh Tappeh  Golestan

[121 rows x 3 columns]

In [12]:
# Save the dataframe to a csv file
df.to_csv('city-translated.csv', index=False)

In [28]:
# Remove .DS_Store from ('./mapped-data/Mazandaran/Daily')
os.system('rm ./mapped-data/Golestan/Daily/.DS_Store')

0

In [44]:
# Get name of files in ./mapped-data/Gilan/Daily
files = os.listdir('./mapped-data/Mazandaran/Daily')

# Create a directed graph
G = nx.DiGraph()

# For each file in files
for file in files:
    # Read the name
    name = re.sub(r'.csv', '', file)
    print(name)
    # if file has parentheses
    first_part = name
    second_part = ''
    if '(' in name:
        # Split by parentheses
        name = name.split('(')
        # Get the first element
        first_part = name[0].strip()
        second_part = name[1].strip()
        # Remove )
        second_part = second_part[:-1]
    # Split by -
    first_part = first_part.split('-')
    first_name = first_part[0].strip()
    second_name = first_part[1].strip()

    # If first_name is not in the graph
    if translation[first_name] not in G:
        # Add the node
        G.add_node(translation[first_name])
    # # If second_name is not in the graph
    if translation[second_name] not in G:
        # Add the node
        G.add_node(translation[second_name])
    # if first_name not in G:
    #     G.add_node(first_name)
    # if second_name not in G:
    #     G.add_node(second_name)

    # Read the file
    df = pd.read_csv(f'./mapped-data/Mazandaran/Daily/{file}')
    # Find a row that has [زمان شروع] contains year/month/day
    row = df[df['زمان شروع'].str.contains(f'{year}/{month}/{day}', na=False)]
    # If the row is empty set value to 0
    # if row.shape[0] == 0:
    #     value = 0
    # else:
    #     value = row['تعداد کل وسیله نقلیه'].values[0]
    # # If second_part is ' '
    # if second_part == '':
    #     label = f'{value}'
    # else:
    #     label = f'{second_part} {value}'
    # G.add_edge(translation[first_name], translation[second_name], weight=np.log10(value + 0.001), label=second_part)
    G.add_edge(translation[first_name], translation[second_name], label=second_part)

    # G.add_edge(first_name, second_name, label=label)
    # G.add_edge(first_name, second_name, label=second_part)

کياکلا - بابل
شاه زيد - سه راه سنگچال (رودهن - آمل)
کياسر - ساري (امره)
بابلسر - بهنمير
بهشهر - بندر اميرآباد (رستمکلا)
گلوگاه - بهشهر (گلستان)
نکا - بهشهر
دابودشت - فريدونکنار
آمل - پارک جنگلي (آمل - رودهن)
بندر اميرآباد - بهشهر (رستمکلا)
ساري - کياسر (امره)
بابل - گتاب
بهشهر - گلوگاه (گلستان)
بهنمير - بابلسر
شيرگاه - پل سفيد (قائمشهر - فيروزکوه)
گدوک - ورسک (فيروزکوه - قائمشهر)
کياسر - ساري (کیاده)
پارک جنگلي - سه راه سنگچال (آمل - رودهن)
بابل - کياکلا
نوشهر - چالوس
دوآب - مرزن آباد (چالوس - کرج)
مرزن آباد - چالوس (تهران - شمال)
سوادکوه - قائمشهر
بهشهر - گلوگاه
کلاردشت - عباس آباد
نوشهر - سی سنگان (نوشهر - نور)
پنجاب - گزنک (آمل - رودهن)
بابل - آمل (بزرگراه)
عباس آباد - کلارآباد
چالوس - مرزن آباد (تهران - شمال)
آمل - بابل (قدیم)
جويبار - قائمشهر
ورسک - گدوک (قائمشهر - فيروزکوه)
کندوان - سياه بيشه (کرج - چالوس)
فريدونکنار - بابلسر
نور - محمودآباد
گرزين خيل - پل سفيد (فيروزکوه - قائمشهر)
تنکابن - رامسر
مرزن آباد - مکارود (چالوس - کرج)
سمنان - کياسر
نکا - زاغمرز (حسين آباد)
گرزين خيل -

In [45]:
import json

with open('coordinates.json') as f:
    coordinates = json.load(f)

In [46]:
nt = Network('1500px', '1000px', directed=True, select_menu=True, filter_menu=True, notebook=False)
nt.from_nx(G)

for node in nt.nodes:
    outgoing = 0
    for edge in nt.edges:
        print(edge)
    print(node)

# Set the weight of all nodes to their incoming weights - minuts their outgoing weights
# for node in nt.nodes:
#     outgoing = 0
#     for edge in nt.edges:
#         # If the edge label contains ' '
#         if ' ' in edge['label']:
#             # Split by space
#             temp = edge['label'].split(' ')
#             weight = float(temp[-1])
#         else:
#             weight = float(edge['label'])
#         if edge['to'] == node['id']:
#             outgoing += weight
#         if edge['from'] == node['id']:
#             outgoing -= weight
#     # node['label'] = node['id'] + ' ' + str(outgoing)
#     # print(node['label'])
#     node['size'] = np.log10(abs(outgoing) + 0.001)
#     if(outgoing < 0):
#         node['color'] = 'red'
#     elif outgoing > 0:
#         node['color'] = 'green'
#     else:
#         node['color'] = 'blue'

{'label': '', 'width': 1, 'from': 'Kiyakala', 'to': 'Babol', 'arrows': 'to'}
{'label': '', 'width': 1, 'from': 'Babol', 'to': 'Gatab', 'arrows': 'to'}
{'label': '', 'width': 1, 'from': 'Babol', 'to': 'Kiyakala', 'arrows': 'to'}
{'label': 'قدیم', 'width': 1, 'from': 'Babol', 'to': 'Amol', 'arrows': 'to'}
{'label': '', 'width': 1, 'from': 'Babol', 'to': 'Qaemshahr', 'arrows': 'to'}
{'label': '', 'width': 1, 'from': 'Babol', 'to': 'Fereydunkenar', 'arrows': 'to'}
{'label': '', 'width': 1, 'from': 'Babol', 'to': 'Babolsar', 'arrows': 'to'}
{'label': 'رودهن - آمل', 'width': 1, 'from': 'Shahzeyd', 'to': 'Se-rah-e Sangchal', 'arrows': 'to'}
{'label': 'آمل - رودهن', 'width': 1, 'from': 'Shahzeyd', 'to': 'Se-rah-e Baladeh', 'arrows': 'to'}
{'label': 'آمل - رودهن', 'width': 1, 'from': 'Se-rah-e Sangchal', 'to': 'Shahzeyd', 'arrows': 'to'}
{'label': 'رودهن - آمل', 'width': 1, 'from': 'Se-rah-e Sangchal', 'to': 'Park Jangali', 'arrows': 'to'}
{'label': 'کیاده', 'width': 1, 'from': 'Kiasar', 'to':

In [47]:
nt.toggle_physics(True)
# Open physics menu
nt.show_buttons(filter_=['physics'])
# nt.force_atlas_2based(spring_length=300)
nt.show('example.html', notebook=False)
os.system('open example.html')

example.html


0

In [48]:
for edge in nt.edges:
    if edge['label'] != '':
        print(edge['label'])

قدیم
رودهن - آمل
آمل - رودهن
آمل - رودهن
رودهن - آمل
کیاده
کياسر - ساري
کیاده
ساري - کياسر
رستمکلا
رستمکلا
گلوگاه
حسين آباد
آمل - رودهن
آمل - بابل
آمل - رودهن
رودهن - آمل
قائمشهر - فيروزکوه
فيروزکوه - قائمشهر
قائمشهر - فيروزکوه
فيروزکوه - قائمشهر
فيروزکوه - قائمشهر
فيروزکوه - قائمشهر
فيروزکوه - قائمشهر
نوشهر - نور
تهران - شمال
چالوس - کرج
چالوس - کرج
کرج - چالوس
تهران - شمال
چالوس - کرج
کرج - چالوس
قائمشهر - فيروزکوه
آمل - بابل
نوشهر - نور
نور - نوشهر
آمل - رودهن
رودهن - آمل
رودهن - آمل
کرج - چالوس
چالوس - کرج
کرج - چالوس
نور - نوشهر
فيروزکوه - قائمشهر
قائمشهر - فيروزکوه
کرج - چالوس
چالوس - کرج
حسين آباد
گلوگاه
آمل - رودهن
رودهن - آمل
رودهن - آمل
آمل - رودهن
کرج - چالوس
چالوس - کرج
ساري - کياسر
کياسر - ساري


In [49]:
road_translation = {
  'آستارا - اردبيل': 'Astara-Ardabil Rd',
  'رشت - قزوين': 'Rasht-Ghazvin Rd',
  'قزوين - رشت': 'Ghazvin-Rasht Rd',
  'آزادراه': 'FWY',
  'قدیم': 'Old Rd',
  'فومن': 'Fuman Rd',
  'سراوان': 'Saravan Rd',
  'مرز استان': 'Gilan Exit Rd',
  'اسالم': 'Asalem Rd',
  'منجيل - طارم': 'Manjil-Tarom Rd',
  'شلمان': 'Shalman Rd',
  'طارم - منجيل': 'Tarom-Manjil Rd',
  'اردبيل - آستارا': 'Ardabil-Astara Rd',
  '': '',
}

# Create a dataframe using the translation dictionary
df = pd.DataFrame(list(road_translation.items()), columns=['Persian', 'English'])
df['Province'] = 'Gilan'

In [50]:
mazandaran_roads = {
'رودهن - آمل': 'Rudhen-Amol Rd',
'آمل - رودهن': 'Amol-Rudhen Rd',
'کياسر - ساري': 'Kiasar-Sari Rd',
'کیاده': 'Kiyadeh Rd',
'ساري - کياسر': 'Sari-Kiasar Rd',
'رستمکلا': 'Rostamkala Rd',
'گلوگاه': 'Galugah Rd',
'حسين آباد': 'Hoseinabad Rd',
'آمل - بابل': 'Amol-Babol Rd',
'آمل - رودهن': 'Amol-Rudhen Rd',
'قائمشهر - فيروزکوه': 'Qaemshahr-Firuzkuh Rd',
'فيروزکوه - قائمشهر': 'Firuzkuh-Qaemshahr Rd',
'نوشهر - نور': 'Nowshahr-Nur Rd',
'تهران - شمال': 'Tehran-North Rd',
'چالوس - کرج': 'Chalus-Karaj Rd',
'کرج - چالوس': 'Karaj-Chalus Rd',
'تهران - شمال': 'Tehran-North Rd',
'نور - نوشهر': 'Nur-Nowshahr Rd',
}

df2 = pd.DataFrame(list(mazandaran_roads.items()), columns=['Persian', 'English'])
df2['Province'] = 'Mazandaran'
df2

Persian                English    Province
0         رودهن - آمل         Rudhen-Amol Rd  Mazandaran
1          آمل - رودهن         Amol-Rudhen Rd  Mazandaran
2         کياسر - ساري         Kiasar-Sari Rd  Mazandaran
3                کیاده             Kiyadeh Rd  Mazandaran
4         ساري - کياسر         Sari-Kiasar Rd  Mazandaran
5              رستمکلا          Rostamkala Rd  Mazandaran
6               گلوگاه             Galugah Rd  Mazandaran
7            حسين آباد          Hoseinabad Rd  Mazandaran
8           آمل - بابل          Amol-Babol Rd  Mazandaran
9   قائمشهر - فيروزکوه  Qaemshahr-Firuzkuh Rd  Mazandaran
10  فيروزکوه - قائمشهر  Firuzkuh-Qaemshahr Rd  Mazandaran
11         نوشهر - نور        Nowshahr-Nur Rd  Mazandaran
12        تهران - شمال        Tehran-North Rd  Mazandaran
13         چالوس - کرج        Chalus-Karaj Rd  Mazandaran
14         کرج - چالوس        Karaj-Chalus Rd  Mazandaran
15         نور - نوشهر        Nur-Nowshahr Rd  Mazandaran

In [51]:
golestan_roads = {
'آزادشهر - شاهرود': 'Azadshahr-Shahroud',
'شاهرود - آزادشهر': 'Shahroud-Azadshahr',
'متناظر سامانه توزين': 'ُTowzin Rd',
'توسکستان': 'Tuskestan Rd'
}

df3 = pd.DataFrame(list(golestan_roads.items()), columns=['Persian', 'English'])
df3['Province'] = 'Golestan'
df3

Persian             English  Province
0     آزادشهر - شاهرود  Azadshahr-Shahroud  Golestan
1     شاهرود - آزادشهر  Shahroud-Azadshahr  Golestan
2  متناظر سامانه توزين          ُTowzin Rd  Golestan
3             توسکستان        Tuskestan Rd  Golestan

In [52]:
# Append df, df2, and df3
df = pd.concat([df, df2, df3], ignore_index=True)
df

Persian                English    Province
0       آستارا - اردبيل      Astara-Ardabil Rd       Gilan
1           رشت - قزوين       Rasht-Ghazvin Rd       Gilan
2           قزوين - رشت       Ghazvin-Rasht Rd       Gilan
3               آزادراه                    FWY       Gilan
4                  قدیم                 Old Rd       Gilan
5                  فومن               Fuman Rd       Gilan
6                سراوان             Saravan Rd       Gilan
7             مرز استان          Gilan Exit Rd       Gilan
8                 اسالم              Asalem Rd       Gilan
9          منجيل - طارم        Manjil-Tarom Rd       Gilan
10                شلمان             Shalman Rd       Gilan
11         طارم - منجيل        Tarom-Manjil Rd       Gilan
12      اردبيل - آستارا      Ardabil-Astara Rd       Gilan
13                                                   Gilan
14         رودهن - آمل         Rudhen-Amol Rd  Mazandaran
15          آمل - رودهن         Amol-Rudhen Rd  Mazandaran
16         کياسر - ساري         Kiasar-Sari Rd  Mazandaran
17                کیاده             Kiyadeh Rd  Mazandaran
18         ساري - کياسر         Sari-Kiasar Rd  Mazandaran
19              رستمکلا          Rostamkala Rd  Mazandaran
20               گلوگاه             Galugah Rd  Mazandaran
21            حسين آباد          Hoseinabad Rd  Mazandaran
22           آمل - بابل          Amol-Babol Rd  Mazandaran
23   قائمشهر - فيروزکوه  Qaemshahr-Firuzkuh Rd  Mazandaran
24   فيروزکوه - قائمشهر  Firuzkuh-Qaemshahr Rd  Mazandaran
25          نوشهر - نور        Nowshahr-Nur Rd  Mazandaran
26         تهران - شمال        Tehran-North Rd  Mazandaran
27          چالوس - کرج        Chalus-Karaj Rd  Mazandaran
28          کرج - چالوس        Karaj-Chalus Rd  Mazandaran
29          نور - نوشهر        Nur-Nowshahr Rd  Mazandaran
30     آزادشهر - شاهرود     Azadshahr-Shahroud    Golestan
31     شاهرود - آزادشهر     Shahroud-Azadshahr    Golestan
32  متناظر سامانه توزين             ُTowzin Rd    Golestan
33             توسکستان           Tuskestan Rd    Golestan

In [53]:
# Save to a csv file
df.to_csv('road-translated.csv', index=False)

In [633]:
# Get ID of all the nodes
for node in nt.nodes:
    print(node['id'])

کردکوي
بندر ترکمن
گرگان
گلوگاه
سه راهي يساقي
آق قلا
گميشان
گنبد
مينودشت
آزادشهر
اينچه برون
سه راهي کلاله
گاليکش
خوش ییلاق
علي آباد
شاهرود
تنگراه
جنگل گلستان
کلاله
مرزپل
مراوه تپه


In [ ]:
translation_mazandaran = {
  'کياکلا': 'Kiakola',
  'بابل': 'Babol',
  'گتاب': 'Gatab',
  'آمل': 'Amol',
  'قائمشهر': 'Qaemshahr',
  'فريدونکنار': 'Fereydunkenar',
  'بابلسر': 'Babolsar',
  'شاه زيد': 'Shahzeyd',
  'سه راه سنگچال': 'Se-rah-e Sangchal',
  'سه راه بلده': 'Se-rah-e Baladeh',
  'پارک جنگلي': 'Park Jangali',
  'کياسر': 'Kiasar',
  'ساري': 'Sari',
  'گرماب': 'Garmab',
  'سمنان': 'Semnan',
  'جويبار': 'Juybar',
  'نکا': 'Neka',
  'خزرآباد': 'Khazarabad',
  'شکتا': 'Shakta',
  'بهنمير': 'Behnamir',
  'زرگرشهر': 'Zargarshahr',
  'بهشهر': 'Behshahr',
  'بندر اميرآباد': 'Bandar-e Amirabad',
  'گلوگاه': 'Galugah',
  'کردکوي': 'Kordkuy',
  'زاغمرز': 'Zagheh Marz',
  'دابودشت': 'Dabudsht',
  'محمودآباد': 'Mahmudabad',
  'شيرگاه': 'Shirgah',
  'پل سفيد': 'Pol-e Sefid',
  'گرزين خيل': 'Garzin Kheil',
  'گدوک': 'Gaduk',
  'ورسک': 'Veresk',
  'نوشهر': 'Nowshahr',
  'چالوس': 'Chalus',
  'سی سنگان': 'Sisangan',
  'مرزن آباد': 'Marzanabad',
  'کلارآباد': 'Kalarabad',
  'عباس آباد': 'Abbasabad',
  'دوآب': 'Doab',
  'مکارود': 'Makarud',
  'سوادکوه': 'Savadkuh',
  'فيروزکوه': 'Firuzkuh',
  'کلاردشت': 'Kalardasht',
  'تنکابن': 'Tonekabon',
  'نور': 'Nur',
  'پنجاب': 'Panjab',
  'گزنک': 'Gazanak',
  'پلور': 'Polur',
  'کندوان': 'Kandovan',
  'سياه بيشه': 'Siah Bisheh',
  'هزارچم': 'Hazarcham',
  'رامسر': 'Ramsar',
  'چابکسر': 'Chaboksar',
  'امامزاده هاشم تهران': 'Imamzadeh Hashem Tehran'
}

In [ ]:
translation_golestan = {
'کردکوي': 'Kordkuy',
'بندر ترکمن': 'Bandar Torkaman',
'گرگان': 'Gorgan',
'گلوگاه': 'Galugah',
'سه راهي يساقي': 'Se-rah-e Yasaqi',
'آق قلا': 'Aq Qala',
'گميشان': 'Gomishan',
'گنبد': 'Gonbad',
'مينودشت': 'Minudasht',
'آزادشهر': 'Azadshahr',
'اينچه برون': 'Incheh Borun',
'سه راهي کلاله': 'Se-rah-e Kalaleh',
'گاليکش': 'Galikesh',
'خوش ییلاق': 'Khosh Yeylagh',
'علي آباد': 'Aliabad',
'شاهرود': 'Shahroud',
'تنگراه': 'Tangrah',
'جنگل گلستان': 'Golestan Forest',
'کلاله': 'Kalaleh',
'مرزپل': 'Marzpol',
'مراوه تپه': 'Maraveh Tappeh',
}